<a href="https://colab.research.google.com/github/Rokia88/EMV/blob/master/LSTM_win3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from urllib.parse import unquote
import numpy as np
from sklearn.preprocessing import StandardScaler
import nltk
import re
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import plot_model,to_categorical
# Scikit learn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Word2vec
import gensim
import multiprocessing
# Keras
from tensorflow import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.constraints import maxnorm
from keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
import tensorflow as tf
import os
import math
import keras_metrics
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,SGD
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from gensim.models.callbacks import CallbackAny2Vec

checkpoint_filepath = './checkmodel5/'
datasetfile = './csic3.csv'
word2vec_filepath = './word2vec1.model'
sgd = SGD(lr=0.01,momentum=0.9,decay=0.003, nesterov=True)
adam = Adam()
OPTIMIZER = adam
BATCH_SIZE = 128
NB_EPOCH = 10
# load the dataset
def load_dataset(filename):
    # load the dataset as a pandas DataFrame
    dataset = read_csv(filename,names=['URL','label'], sep=';',header=0)
    # split into input (X) and output (y) variables
    X = dataset
    y = dataset[['label']]
    print(X.shape)
    print(y.shape)
    return X, y

def URLDECODE(payload):
    payload = payload.lower()
    while True:
        test = payload
        payload = unquote(payload)
        if test == payload:
            break
        else:
            continue
    payload, num = re.subn(r'\d+', "0", payload)
    
    payload, num = re.subn(r'(http|https)://[a-zA-Z0-9\.@&/#!#\?]+', "http://u", payload)
    
    r = '''
        (?x)[\w\.]+?\(
        |\)
        |"\w+?"
        |'\w+?'
        |http://\w
        |</\w+>
        |<\w+>
        |<\w+
        |\w+=
        |>
        |[\w\.]+
        '''
    return nltk.regexp_tokenize(payload, r)

# init callback class
class callback(CallbackAny2Vec):
    """
        Callback to print loss after each epoch
        """
    def __init__(self):
        self.epoch = 0
    
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

def listToString(s): 
    # initialize an empty string
    str1 = ""
    # traverse in the string
    for ele in s:
        str1 += ele
    # return string
    return str1

def create_model(embedding_matrix,vocab_size,MAX_LEN):
    # define the model
    # input layer
    initializer = tf.keras.initializers.HeNormal()
    input = tf.keras.Input(shape=(MAX_LEN,))
    embd =  tf.keras.layers.Embedding(vocab_size,128,weights=[embedding_matrix],input_length=MAX_LEN,trainable=False)(input)
    lstm1 = tf.compat.v1.keras.layers.CuDNNLSTM(128,kernel_initializer=initializer)(embd)
    lstm1 = tf.keras.layers.LayerNormalization()(lstm1)
    lstm2= tf.compat.v1.keras.layers.CuDNNLSTM(256,kernel_initializer=initializer)(embd)
    lstm2 = tf.keras.layers.LayerNormalization()(lstm2)
    lstm3= tf.compat.v1.keras.layers.CuDNNLSTM(512,kernel_initializer=initializer)(embd)
    lstm3 = tf.keras.layers.LayerNormalization()(lstm3)
    merge = tf.keras.layers.concatenate([lstm1,lstm2,lstm3])
    merge = tf.keras.layers.LayerNormalization()(merge)
    merge = tf.keras.layers.Dropout(0.5)(merge)
    hidden4 = tf.keras.layers.Dense(1024, activation='tanh',kernel_initializer=initializer)(merge)
    hidden4 = tf.keras.layers.LayerNormalization()(hidden4)
    hidden4 = tf.keras.layers.Dropout(0.5)(hidden4)
    output = tf.keras.layers.Dense(2, activation='softmax')(hidden4)
    model = tf.keras.Model(inputs=[input], outputs=output)
    
    # summarize the model
    print(model.summary())
    tf.keras.utils.plot_model(model, to_file='shared_input_layer.png')
    # compile the model
    model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

def prepare_targets(y_train, y_val, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc1 = le.transform(y_test)
    y_val_enc = le.transform(y_val)
    y_train_enc = to_categorical(y_train_enc, num_classes=2)
    y_test_enc = to_categorical(y_test_enc1, num_classes=2)
    y_val_enc = to_categorical(y_val_enc, num_classes=2)
    return y_train_enc, y_val_enc, y_test_enc,y_test_enc1

def texts_to_sequences(s_tokens,word_index):
    result = []
    l2 = []
    for s in s_tokens:
        for token in s:
            if word_index.get(token) is not None:
                l2.append(word_index.get(token))
            else:
                l2.append(0)
        result.append(l2)
        l2 = []
    print(result[0],result[1])
    if(len(result)==0):
        print("result empty")
    return result

def standardize_data(x_train,x_test,x_val):
    trans = StandardScaler()
    trans.fit(x_train)
    x_train_std = trans.transform(x_train)
    x_test_std = trans.transform(x_test)
    x_val_std = trans.transform(x_val)
    return x_train_std,x_test_std,x_val_std

def find_max_list(list):
    list_len = [len(i) for i in list]
    return (max(list_len))

X, y = load_dataset(datasetfile)
run = 0
loss_avg = 0
acc_avg = 0
prec_avg = 0
reca_avg = 0
acc_min = 1
acc_max = 0
MAX_RUN = 5
while run < MAX_RUN:
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.33,stratify=y,shuffle= True,random_state=20)
        # prepare output data
        #y_train_enc1, y_test_enc= prepare_targets(y_train1, y_test)
        X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.16,stratify=y_temp,shuffle= True,random_state=20)
        # prepare output data
        y_train_enc, y_val_enc, y_test_enc,y_test_enc1= prepare_targets(y_train, y_val,y_test)
        print(X_train.head())
        print(X_train.shape)
        print(X_train.label.value_counts())
        print(X_test.head())
        print(X_test.shape)
        print(X_test.label.value_counts())
        print(X_val.head())
        print(X_val.shape)
        print(X_val.label.value_counts())
        # split text in tokens
        # split text in tokens
        sentences = [URLDECODE(listToString(text)) for text in X.values[:,:-1]]
        model = gensim.models.Word2Vec(alpha=0.1,min_alpha=0.0007,min_count=1, window=3,size=128)
        model.build_vocab(sentences)
        model.train(sentences, total_examples=model.corpus_count, epochs=30,compute_loss = True,callbacks=[callback()])
        #model = gensim.models.Word2Vec.load(word2vec_filepath)
        words = list(model.wv.vocab)
        print("Vocabulary size: %i" % len(words))
        sentences_training = []
        for text in X_train.values[:,:-1]:
            l1 = URLDECODE(listToString(text))
            for u in l1:
                sentences_training.append(u)
        # creating a dictionary of vocab with index
        vocab = sorted(set(sentences_training))
        vocab = list(enumerate(vocab, 1))
        print(len(vocab))
        # putting the index first
        indexed_vocab = {k:v for v,k in dict(vocab).items()}
        print(len(indexed_vocab))
        print("Create Embedding matrix")
        #word_index = tokenizer.word_index
        vocab_size = len(words) + 1
        embedding_matrix = np.zeros((len(words) + 1, 128))
        for word, idx in indexed_vocab.items():
            if word in words:
                embedding_vector = model.wv.get_vector(word)
                if embedding_vector is not None:
                    embedding_matrix[idx] = model.wv[word]
        print("Embedding matrix: %s" % str(embedding_matrix.shape))
        print("Build Keras model")
        sentences_train = [URLDECODE(listToString(text)) for text in X_train.values[:,:-1]]
        x_train= keras.preprocessing.sequence.pad_sequences(texts_to_sequences(sentences_train,indexed_vocab),maxlen=20)
        sentences_val = [URLDECODE(listToString(text)) for text in X_val.values[:,:-1]]
        x_val = keras.preprocessing.sequence.pad_sequences(texts_to_sequences(sentences_val,indexed_vocab),maxlen=20)
        sentences_test = [URLDECODE(listToString(text)) for text in X_test.values[:,:-1]]
        x_test = keras.preprocessing.sequence.pad_sequences(texts_to_sequences(sentences_test,indexed_vocab),maxlen=20)
        print('x_train shape: %s' % str(x_train.shape))
        print(x_train[0,:])
        print('x_val shape: %s' % str(x_val.shape))
        print(x_val[0,:])
        print('x_test shape: %s' % str(x_test.shape))
        print(x_test[0,:])
        checkpoint_filepath = checkpoint_filepath+str(run)+"cp5.ckpt"
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy',
                                                    mode='auto',save_best_only='True',save_weights_only='True',verbose=1)
        #early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='max')
        #rop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1, epsilon=1e-4, mode='max')
        #scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay)
        #callbacks = [early_stopping, rop]
        print(np.any(np.isnan(x_train)))
        print(np.any(np.isnan(y_train_enc)))
        print(np.any(np.isnan(x_val)))
        print(np.any(np.isnan(y_val_enc)))
        print(np.any(np.isnan(x_test)))
        print(np.any(np.isnan(y_test_enc)))
        #trainng = DataFrame(x_train)
        #print(trainng.describe())
        #x_train,x_test,x_val = standardize_data(x_train,x_test,x_val)
        #trainng_2 = DataFrame(x_train)
        #print(trainng_2.describe())
        MAX_LEN = 20
        k_model = create_model(embedding_matrix,vocab_size,MAX_LEN)
        history = k_model.fit(x=x_train,verbose=2,
                y=y_train_enc,
                batch_size=BATCH_SIZE,
                epochs=NB_EPOCH,
                validation_data=(x_val, y_val_enc),
                callbacks=[checkpoint])
        pyplot.subplot(211)
        pyplot.title('Loss')
        pyplot.plot(history.history['loss'], label='train'+str(run))
        pyplot.plot(history.history['val_loss'], label='test'+str(run))
        # plot accuracy during training
        pyplot.subplot(212)
        pyplot.title('Accuracy')
        pyplot.plot(history.history['accuracy'], label='train'+str(run))
        pyplot.plot(history.history['val_accuracy'], label='test'+str(run))
        new_model = create_model(embedding_matrix,vocab_size,MAX_LEN)
        new_model.load_weights(checkpoint_filepath)
        loss, accuracy = new_model.evaluate(x_test,y_test_enc, verbose=0)
        print('RUN:%f'%(run))
        predicted_classes = new_model.predict(x_test)
        print(predicted_classes)
        yhat = np.argmax(predicted_classes, axis=1)
        print(classification_report(y_test_enc1, yhat))
        print('Accuracy: %f, loss:%f' % (accuracy*100,loss))
        acc_avg += accuracy
        loss_avg += loss
        if (accuracy<acc_min):
            acc_min = accuracy
        if (accuracy>acc_max):
            acc_max= accuracy
        run = run +1
acc_avg = acc_avg/MAX_RUN
loss_avg = loss_avg/MAX_RUN
print('Max Accuracy: %f,Min Accuracy: %f,Avg Accuracy: %f, Avg loss:%f' % (acc_max*100,acc_min*100,acc_avg*100,loss_avg))
pyplot.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install keras-metrics
!cp "drive/My Drive/Colab Notebooks/csic3.csv.zip" .
# importing required modules 
from zipfile import ZipFile 

# specifying the zip file name 
file_name = "csic3.csv.zip"

# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
  # printing all the contents of the zip file 
  zip.printdir() 

  # extracting all the files 
  print('Extracting all the files now...') 
  zip.extractall() 
  print('Done!')